License: BSD-3-Clause

Copyright the MNE-Python contributors.

Edited by: Edouard Delaire, Ilian Azz

# Install PIP dependencies

In [ ]:
# Able to export maps as movies (optional)
%pip install imageio
# to convert the notebook to html, pdf, etc. (optional)
%pip install nbconvert
%pip install scipy
%pip install mne==1.7.0
%pip install matplotlib
%pip install numpy
%pip install pandas
%pip install darkdetect
# HERE 
# !!!!! The matlabengine version should match the version of MATLAB installed on your system !!!!!
# Check : https://www.mathworks.com/help/matlab/matlab_external/install-the-matlab-engine-for-python.html#
#%pip install matlabengine==23.2.3

# Load MNE model

In [ ]:
import mne

from mne.datasets import sample

data_path = sample.data_path()
meg_path = data_path / "MEG" / "sample"
fwd_fname = meg_path / "sample_audvis-meg-eeg-oct-6-fwd.fif"
ave_fname = meg_path / "sample_audvis-ave.fif"
cov_fname = meg_path / "sample_audvis-shrunk-cov.fif"
subjects_dir = data_path / "subjects"
condition = "Left Auditory"

# Read noise covariance matrix
noise_cov = mne.read_cov(cov_fname)
# Handling average file
evoked = mne.read_evokeds(ave_fname, condition=condition, baseline=(None, 0))
evoked.crop(tmin=0.04, tmax=0.18)

evoked = evoked.pick(picks="meg", exclude="bads")
# Handling forward solution
forward = mne.read_forward_solution(fwd_fname)

# Convertir à un modèle à orientation libre
# force_fixed: Orientation fixe ou libre
# surf_ori: orientaton vers la surface
forward = mne.convert_forward_solution(forward, force_fixed=True, surf_ori=True)

# Call MEM using the matlab wrapper


In [ ]:
# Start MATLAB engine
import bestpython

# Initialize MatlabWrapper
matlab_wrapper = bestpython.MatlabWrapper("/Users/edelaire1/.brainstorm/plugins/brainentropy/best-brainstorm-master")

# Define option for MEM
mem_options = bestpython.MEMOptions(
    data_modality="MEG",
    time_segment = [0.04, 0.18],
    baseline_time = [0.04, 0.1]
)

# Call MEM solver
stc_cmem = matlab_wrapper.mem_solver(evoked, forward, noise_cov, MEMOptions=mem_options)

# Show results

## Brain Map of the solutions

In [ ]:
initial_time = -0.1
brain_cmem = stc_cmem.plot(
    subjects_dir=subjects_dir,
    initial_time=initial_time,
    # clim=dict(kind="value", lims=[0, 5e-11, 1e-10]),
    smoothing_steps=7,
    hemi='both',
)

brain_cmem.add_text(0.1, 0.9, "cMEM", "title", font_size=14)

## Amplitude for each source and vetex

## Compare with MNE Default method

In [ ]:
from mne import read_source_estimate
from mne.minimum_norm import apply_inverse, make_inverse_operator

fname_stc = meg_path / "sample_audvis-meg"

inverse_operator = make_inverse_operator(
    evoked.info, forward, noise_cov, loose=0, depth=0
)

stc_mne = apply_inverse(
    evoked,
    inverse_operator,
    pick_ori=None,
    method="MNE"
)

# Define plotting parameters
surfer_kwargs = dict(
    hemi="both",
    subjects_dir=subjects_dir,
    smoothing_steps=7,
)

# Plot surface
brain_mne = stc_mne.plot(**surfer_kwargs)
brain_mne.add_text(0.1, 0.9, "MNE", "title", font_size=14)

## Show maps with absolute values

In [ ]:
import numpy as np

stc_cmem_abs = stc_cmem.copy()
stc_cmem_abs.data = np.abs(stc_cmem_abs.data)

brain_cmem_abs = stc_cmem_abs.plot(
    subject="sample",
    subjects_dir=subjects_dir,
    initial_time=initial_time,
    smoothing_steps=7,
    hemi='both',
    title="cMEM abs",
)

brain_cmem_abs.add_text(0.1, 0.9, "cMEM abs", "title", font_size=14)

stc_mne_abs = stc_mne.copy()
stc_mne_abs.data = np.abs(stc_mne_abs.data)

# Plot the result
brain_mne_abs = stc_mne_abs.plot(
    subject="sample",
    subjects_dir=subjects_dir,
    initial_time=initial_time,
    smoothing_steps=7,
    hemi='both',
)

brain_mne_abs.add_text(0.1, 0.9, "MNE abs", "title", font_size=14)